## Description

Let's read in the summary statistics for the CAD dataset:
- File input: `GWAS_MDD.txt`.

**Data Cleaning steps:**
1. Remove one row with NaNs
2. Type cast columns `CHR` and `BP` as type: `int`
3. Rename column: `SE` to `SE_BETA`
4. Compute columns: `BETA` and `Z` 
5. Subset to a new dataframe to include relevant columns
6. Sort dataframe by `(CHR, BP)`
7. Write out cleaned dataset to `.csv` file format

**Expected final cleaned dataset columns:**
1. `CHR`
2. `SNP`
3. `BP`
4. `A1`
5. `A2`
6. `OR`
7. `BETA`
8. `SE_BETA`
9. `P`
10. `Z`
11. `MAF`

In [1]:
import numpy as np
import pandas as pd

df2 = pd.read_csv("/gpfs/gibbs/project/bdsi/shared/Genetics/data/real_data/GWAS_MDD.txt", sep=" ")

In [2]:
# Apply the logic row-wise
def compute_maf(row):
    return (row['FRQ_A_166773']*row['Nca'] + row['FRQ_U_507679']*row['Nco'])/(row['Nca'] + row['Nco'])
    # if row['FRQ_A_166773'] < row['FRQ_U_507679']:
    #     return row['A1'], row['A2'], row['FRQ_A_166773']
    # else:                                                             
    #     return row['A2'], row['A1'], row['FRQ_U_507679']
                                                                                
# Create MAF and possibly updated alleles
df2[['MAF']] = df2.apply(
    lambda row: pd.Series(compute_maf(row)),
    axis=1
)

In [3]:
df2.head()

,CHR,SNP,BP,A1,A2,FRQ_A_166773,FRQ_U_507679,INFO,OR,SE,P,ngt,Direction,HetISqt,HetDf,HetPVa,Nca,Nco,Neff_half,MAF
0,13.0,rs10161922,110122582.0,A,G,0.873,0.872,0.999,1.00793,0.0062,0.2041,0,+++-,3.4,3.0,0.375800,166773.0,507679.0,226782.08,0.872247
1,13.0,rs893213,112770951.0,C,A,0.101,0.115,0.955,1.01025,0.0072,0.1605,1,+---,9.2,3.0,0.347200,166773.0,507679.0,226782.08,0.111538
2,13.0,rs9588302,111581488.0,T,C,0.944,0.951,0.970,1.00833,0.0095,0.3798,0,+-++,75.0,3.0,0.007437,166773.0,507679.0,226782.08,0.949269
3,13.0,rs9588059,110647877.0,T,C,0.884,0.881,0.998,0.99700,0.0066,0.6507,0,--++,0.0,3.0,0.750800,166773.0,507679.0,226782.08,0.881742
4,13.0,rs7323548,113272878.0,G,A,0.923,0.903,0.993,1.00210,0.0085,0.8029,26,++--,35.0,3.0,0.202000,166773.0,507679.0,226782.08,0.907945


In [4]:
## Step 1: Remove one row with NaNs

# Check rows with NaN values in "CHR", "SNP", "BP"
df2[df2[['SNP', 'CHR', 'BP']].isna().any(axis=1)]

# Remove that 1 row
df2 = df2[~df2[['SNP', 'CHR', 'BP']].isna().any(axis=1)]

In [5]:
len(df2)

7266505

In [6]:
## Step 2: Type cast "CHR" and "BP" as int
df2['CHR'] = df2['CHR'].astype(int)
df2['BP'] = df2['BP'].astype(int)

# Step 3: Rename 'SE' to 'SE_BETA'
df2 = df2.rename(columns={'SE': 'SE_BETA'})

# Step 4: Compute BETA from OR; Compute Z from BETA and SE_BETA
df2['BETA'] = np.log(df2['OR'])
df2['Z'] = df2['BETA'] / df2['SE_BETA']

## Step 5: Subset to a new dataframe
#N_value = 674452
df2_new = pd.DataFrame({
    'CHR': df2['CHR'],
    'SNP': df2['SNP'],
    'BP': df2['BP'],
    'A1': df2['A1'],
    'A2': df2['A2'],
    'OR': df2['OR'],
    'BETA': df2['BETA'],
    'SE_BETA': df2['SE_BETA'],
    'P': df2['P'],
    'Z': df2['Z'],
    'MAF': df2['MAF']
    #'N': N_value
})

## Step 6: Sort by CHR then BP
df2_new = df2_new.sort_values(by=['CHR', 'BP'], na_position='last').reset_index(drop=True)

In [7]:
df2_new.head()

,CHR,SNP,BP,A1,A2,OR,BETA,SE_BETA,P,Z,MAF
0,1,rs12238997,693731,A,G,0.99900,-0.001001,0.0107,0.9284,-0.093505,0.843911
1,1,rs4951859,729679,C,G,0.99790,-0.002102,0.0097,0.8311,-0.216722,0.193262
2,1,rs148120343,730087,T,C,0.98423,-0.015896,0.0171,0.3532,-0.929571,0.944587
3,1,rs142557973,731718,T,C,1.00030,0.000300,0.0102,0.9760,0.029407,0.836911
4,1,rs141242758,734349,T,C,0.99900,-0.001001,0.0102,0.9207,-0.098088,0.838911


In [8]:
#print("MDD dataset data cleaning done.")

In [9]:
dat = pd.read_csv("/gpfs/gibbs/project/bdsi/bdsi_kc2587/lo/new/1_CAD/CAD_clean.csv")

In [10]:
# Merge the two dataframes on 'SNP'
merged = pd.merge(dat[['SNP', 'A1']], df2_new[['SNP', 'A1']], on='SNP', suffixes=('_CAD', '_MDD'))

# Count mismatches
mismatch = (merged['A1_CAD'] != merged['A1_MDD'])
mismatch_count = mismatch.sum()

print(f"Number of shared SNPs with mismatched A1 alleles: {mismatch_count}")

Number of shared SNPs with mismatched A1 alleles: 1556354


In [11]:
merged.to_csv("merged.csv", index=False)

In [12]:
# Merge the two dataframes on 'SNP'
import pandas as pd
merged = pd.read_csv('merged.csv')

# Count mismatches
mismatch = (merged['A1_CAD'] != merged['A1_MDD'])
mismatch_count = mismatch.sum()

print(f"Number of shared SNPs with mismatched A1 alleles: {mismatch_count}")

Number of shared SNPs with mismatched A1 alleles: 1556354


In [13]:
# indx = merged['SNP'][mismatch]

# for snp in indx:
#     row = df2_new.index[df2_new['SNP'] == snp][0]
#     df2_new.loc[row, ['A1', 'A2']] = df2_new.loc[row, ['A2', 'A1']].values
#     df2_new.loc[row, 'OR'] = 1 / df2_new.loc[row, 'OR']
#     df2_new.loc[row, 'BETA'] = -df2_new.loc[row, 'BETA']


In [15]:
# Find all rows in df2_new where SNP is in indx
indx = merged['SNP'][mismatch]
mask = df2_new['SNP'].isin(indx)

# Swap A1 and A2
df2_new.loc[mask, ['A1', 'A2']] = df2_new.loc[mask, ['A2', 'A1']].values

# Invert OR
df2_new.loc[mask, 'OR'] = 1 / df2_new.loc[mask, 'OR']

# Negate BETA
df2_new.loc[mask, 'BETA'] = -df2_new.loc[mask, 'BETA']


In [16]:
## Step 7: Write cleaned CAD dataset to .csv file
df2_new.to_csv("MDD_clean.csv", index=False)